In [1]:
import os
import cv2
import time
import torch
import lpips
import numpy as np
from dggt.models.vggt import VGGT
from dggt.utils.pose_enc import pose_encoding_to_extri_intri
from dggt.utils.geometry import unproject_depth_map_to_point_map
from dggt.utils.gs import concat_list, get_split_gs

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


In [2]:
def alpha_t(t, t0, alpha, gamma0 = 1, gamma1 = 0.1):
    sigma = torch.log(torch.tensor(gamma1)).to(gamma0.device) / ((gamma0)**2 + 1e-6)
    conf = torch.exp(sigma*torch.tensor(t0-t).to(gamma0.device)**2)
    alpha_ = alpha * conf
    return alpha_.float()

In [3]:
def extract_frames_from_video(video_path, max_frames=100):
    """Extract frames from video"""
    cap = cv2.VideoCapture(video_path)
    frames = []
    total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    interval = max(1, total_frames // max_frames)
    
    frame_count = 0
    while cap.isOpened() and len(frames) < max_frames:
        ret, frame = cap.read()
        if not ret:
            break
        
        if frame_count % interval == 0:
            frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
            frames.append(frame_rgb)
        
        frame_count += 1
    
    cap.release()
    return frames

def save_ply(filename, pts3d, view):
    # 🧹 过滤非法数值
    valid_mask = np.isfinite(pts3d).all(axis=1)
    valid_mask &= np.isfinite(view).all(axis=1)
    pts3d = pts3d[valid_mask]
    view = view[valid_mask]

    with open(filename, 'w') as f:
        f.write('ply\nformat ascii 1.0\n')
        f.write(f'element vertex {len(pts3d)}\n')
        f.write('property float x\nproperty float y\nproperty float z\n')
        f.write('property uchar red\nproperty uchar green\nproperty uchar blue\n')
        f.write('end_header\n')
        for i, p in enumerate(pts3d):
            c = view[i]
            f.write(f'{p[0]} {p[1]} {p[2]} {c[0]} {c[1]} {c[2]}\n')

In [7]:
image_dir = "davis/bear"
input_views = 1
ckpt_path = "checkpoints/model_latest_waymo.pt"
output_path = "result/test/"
output_images = True
output_depth = True
output_metrics = True

device = "cuda:2" if torch.cuda.is_available() else "cpu"

In [5]:
from datasets.dataset import load_and_preprocess_images

image_path_list = []

for root, dirs, files in os.walk(image_dir):
    for file in sorted(files):
        file = os.path.join(root, file)
        image_path_list.append(file)

images = load_and_preprocess_images(image_path_list)    # [T, C, H, W]

bg_mask = np.ones((1, images.shape[0], images.shape[2], images.shape[3]), dtype=bool)   # [B, T, H, W]
bg_mask = torch.from_numpy(bg_mask).to(device)

In [6]:
dtype = torch.float32
loss_fn = lpips.LPIPS(net='alex').to(device)

# 加载模型 DGGT
model = VGGT().to(device)
checkpoint = torch.load(ckpt_path, map_location="cpu")
model.load_state_dict(checkpoint, strict=True)

model.eval()
psnr_list, ssim_list, lpips_list = [], [], []
inference_time_list = []
scene_idx = 1

Setting up [LPIPS] perceptual loss: trunk [alex], v[0.1], spatial [off]


/home/wangpeifeng/miniconda3/envs/dggt/lib/python3.10/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/home/wangpeifeng/miniconda3/envs/dggt/lib/python3.10/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=AlexNet_Weights.IMAGENET1K_V1`. You can also use `weights=AlexNet_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


Loading model from: /home/wangpeifeng/miniconda3/envs/dggt/lib/python3.10/site-packages/lpips/weights/v0.1/alex.pth


/home/wangpeifeng/miniconda3/envs/dggt/lib/python3.10/site-packages/lpips/lpips.py:107: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  self.load_state_dict(torch.load(model_p

In [ ]:
start_time = time.time()

pts3d_all, rgbs_all = [], []

with torch.no_grad():
    with torch.amp.autocast(dtype=dtype, device_type=device):
        predictions = model(images.to(device))              # B, T, C, H, W
        T, _, H, W = images.shape
        extrinsics, intrinsics = pose_encoding_to_extri_intri(predictions['pose_enc'], (H, W))
        extrinsic = extrinsics[0]
        bottom = torch.tensor([0.0, 0.0, 0.0, 1.0], device=extrinsic.device).view(1, 1, 4).expand(extrinsic.shape[0], 1, 4)
        extrinsic = torch.cat([extrinsic, bottom], dim=1)
        intrinsic = intrinsics[0]
        views=input_views

        use_depth = True
        if use_depth:
            depth_map = predictions["depth"][0]
            point_map = unproject_depth_map_to_point_map(depth_map, extrinsics[0], intrinsics[0])[None,...]
            point_map = torch.from_numpy(point_map).to(device).float()
        else:
            point_map = predictions["world_points"]
        gs_map = predictions["gs_map"]
        gs_conf = predictions["gs_conf"]
        dy_map = predictions["dynamic_conf"].squeeze(-1)    # B, T, H, W

        # static prediction
        static_mask = (dy_map < 0.5)
        static_points = point_map[static_mask].reshape(-1, 3)
        gs_dynamic_list = dy_map[static_mask].sigmoid()
        static_rgbs, static_opacity, static_scales, static_rotations = get_split_gs(gs_map, static_mask)
        static_opacity = static_opacity * (1 - gs_dynamic_list)
        static_gs_conf = gs_conf[static_mask]

        # 让每个有效像素点都能够对应到时间戳上
        frame_idx = torch.nonzero(static_mask, as_tuple=False)[:, 1].cpu().numpy()  # B, T, H, W -> N, 4 -> N,  (取第二维度 T)
        
        start_idx = 0
        indices = [start_idx + i for i in range(T)]
        
        timestamps = np.array(indices) - start_idx
        timestamps = timestamps / timestamps[-1] * (T / 4)
        
        gs_timestamps = timestamps[frame_idx]
        
        # dynamic prediction
        dynamic_points, dynamic_rgbs, dynamic_opacitys, dynamic_scales, dynamic_rotations = [], [], [], [], []
        for i in range(dy_map.shape[1]):                    # shape[1] for B, T, C, H, W
            point_map_i = point_map[:, i]                   # B, T, H, W, 3 -> B, H, W, 3
            bg_mask_i = bg_mask[:, i]
            dy_conf_i = dy_map[:, i].sigmoid()

            # points gs_attr 直接使用 bg_mask
            dynamic_point = point_map_i[bg_mask_i].reshape(-1, 3)
            dynamic_rgb, dynamic_opacity, dynamic_scale, dynamic_rotation = get_split_gs(gs_map[:, i], bg_mask_i)
            # 只有计算不透明度的时候才用到 dy_map
            gs_dynamic_list_i = dy_map[:, i][bg_mask_i].sigmoid()
            dynamic_opacity = dynamic_opacity * gs_dynamic_list_i

            dynamic_points.append(dynamic_point)
            dynamic_rgbs.append(dynamic_rgb)
            dynamic_opacitys.append(dynamic_opacity)
            dynamic_scales.append(dynamic_scale)
            dynamic_rotations.append(dynamic_rotation)

        chunked_renders, chunked_alphas = [], []  
        for idx in range(dy_map.shape[1]):
            t0 = timestamps[idx]
            static_opacity_ = alpha_t(gs_timestamps, t0, static_opacity, gamma0 = static_gs_conf)
            static_gs_list = [static_points, static_rgbs, static_opacity_, static_scales, static_rotations]
            '''
            if dynamic_points:
                world_points, rgbs, opacity, scales, rotation = concat_list(
                    static_gs_list,
                    [dynamic_points[idx], dynamic_rgbs[idx], dynamic_opacitys[idx], dynamic_scales[idx], dynamic_rotations[idx]]
                )
            else:
                world_points, rgbs, opacity, scales, rotation = static_gs_list
            '''
            if dynamic_points is None:
                continue
            
            world_points, rgbs, opacity, scales, rotation = static_gs_list
            # world_points, rgbs, opacity, scales, rotation = [dynamic_points[idx], dynamic_rgbs[idx], dynamic_opacitys[idx], dynamic_scales[idx], dynamic_rotations[idx]]

            # 将 world_points 和 rgb 拿出来就好了
            pts3d_all.append(world_points.cpu().numpy())
            rgbs_all.append((rgbs * 255).cpu().numpy())

/data/raid5/wangpeifeng/project/4DReconstruction/dggt/dggt/models/vggt.py:61: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.amp.autocast(enabled=False):


In [9]:
import open3d as o3d
def save_ply_open3d(filename, pts3d, view):
    pts = np.asarray(pts3d).reshape(-1,3)
    colors = np.asarray(view).reshape(-1, view.shape[-1])
    if colors.shape[1] == 4:
        colors = colors[:, :3]
    # open3d expects colors float in [0,1]
    if colors.dtype != np.float32 and colors.dtype != np.float64:
        colors = colors.astype(np.float32) / 255.0
    else:
        if colors.max() > 1.5:
            colors = colors / 255.0
    pc = o3d.geometry.PointCloud()
    pc.points = o3d.utility.Vector3dVector(pts)
    pc.colors = o3d.utility.Vector3dVector(colors)
    o3d.io.write_point_cloud(filename, pc, write_ascii=True)

In [10]:
os.makedirs(output_path, exist_ok=True)

for idx in range(len(pts3d_all)):
    filename = os.path.join(output_path, f'scene_{scene_idx:03d}_frame_{idx:03d}.ply')
    # save_ply(filename, pts3d_all[idx].reshape(-1, 3), rgbs_all[idx].reshape(-1, 3))
    save_ply_open3d(filename, pts3d_all[idx], rgbs_all[idx])